In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from autoplan.token import OCamlTokenizer
from autoplan.trainer import ClassifierTrainer
from autoplan.vis import plot_accuracy, plot_cm, plot_loss
from autoplan.dataset import TrainVal, build_synthetic_dataset, set_random_seed
from autoplan.generator import ProgramGenerator

from grammars.rainfall.ocaml import Program, Labels

from tqdm.auto import tqdm
from scripts.rainfall_ingest import ingest_dataset
import torch
import seaborn as sns
from iterextras import par_for
from pickle_cache import PickleCache
from torch import nn
import dataclasses

pcache = PickleCache()
device = torch.device('cuda:7')

In [3]:
student_dataset = ingest_dataset('T1')

Skipped 16 programs


In [ ]:
N = 10000
K = 1

def build(i):
    set_random_seed(i)
    return build_synthetic_dataset(
        label_set=Labels, 
        N=N, 
        tokenizer=OCamlTokenizer(), 
        generator=ProgramGenerator(Program()), 
        vocab_index=student_dataset.vocab_index)

synthetic_datasets = par_for(build, range(K), process=False, workers=1)

In [ ]:
model_opts = {
    'model': nn.GRU,
    'hidden_size': 64,
    'embedding_size': 64
}

dataset = synthetic_datasets[0]
trainer = ClassifierTrainer(
    dataset=dataclasses.replace(dataset, class_balance=student_dataset.class_balance), 
    device=device, split=TrainVal(dataset, student_dataset), model_opts=model_opts,
    optim_opts={'lr': 1e-4})

losses, train_eval, val_eval = trainer.train(epochs=10)

In [ ]:
plot_loss(losses)

In [ ]:
plot_accuracy(train_eval)

In [ ]:
plot_accuracy(val_eval)

In [ ]:
val_eval[-1].plot_cm('')

In [66]:
for idx in val_eval[-1].incorrect():
    item = student_dataset.dataset[idx]
    print(item['source'])
    print('True: ', Labels(val_eval[-1].true[idx]))
    print('Pred: ', Labels(val_eval[-1].pred[idx]))
    print('='*30 + '\n\n')

let rainfall (alon : int list) =
  (let rec rainfall_helper (alod : int list) (sum : int) (counter : int) =
     (match alod with
      | [] -> if sum = 0 then 0 else sum / counter
      | hd::tl ->
          if hd = (-999)
          then sum / counter
          else
            if hd < 0
            then rainfall_helper tl sum counter
            else rainfall_helper tl (sum + hd) (counter + 1) : int) in
   rainfall_helper alon 0 0 : int)
let list0 = []
let list1 = [1; 1; 1; 1; (-999)]
let list2 = [1; (-4); 6; 8; (-999); 2; 5; 6; 3]

True:  GeneralRainfallLabels.SingleLoop
Pred:  GeneralRainfallLabels.CleanFirst


let rec rainfall_help (alon : float list) (k : float) =
  (match alon with
   | [] -> (0., k)
   | hd::tl ->
       (match hd with
        | (-999.) -> (0., k)
        | n when n >= 0. ->
            (match rainfall_help tl (k +. 1.) with | (s, n) -> ((hd +. s), n))
        | n when n < 0. -> (match rainfall_help tl k with | (s, n) -> (s, n))) : 
  (float * float))
let rainf